# Introduction to `cf_xarray`

This notebook is a brief introduction to `cf_xarray`'s current capabilities.


In [1]:
import cf_xarray as cfxr
import numpy as np
import xarray as xr

Lets read two datasets.


In [2]:
ds = xr.tutorial.load_dataset("air_temperature")
ds.air.attrs["standard_name"] = "air_temperature"
ds

<xarray.Dataset>
Dimensions:  (lat: 25, lon: 53, time: 2920)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

This one is inspired by POP model output and illustrates how the coordinates
attribute is interpreted. It also illustrates one way of tagging curvilinear
grids for convenient use of `cf_xarray`


In [3]:
from cf_xarray.datasets import popds as pop

pop

<xarray.Dataset>
Dimensions:  (nlat: 20, nlon: 30)
Coordinates:
    TLONG    (nlat, nlon) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    TLAT     (nlat, nlon) float64 2.0 2.0 2.0 2.0 2.0 ... 2.0 2.0 2.0 2.0 2.0
    ULONG    (nlat, nlon) float64 0.5 0.5 0.5 0.5 0.5 ... 0.5 0.5 0.5 0.5 0.5
    ULAT     (nlat, nlon) float64 2.5 2.5 2.5 2.5 2.5 ... 2.5 2.5 2.5 2.5 2.5
  * nlon     (nlon) int64 0 1 2 3 4 5 6 7 8 9 ... 20 21 22 23 24 25 26 27 28 29
  * nlat     (nlat) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
Data variables:
    UVEL     (nlat, nlon) float64 15.0 15.0 15.0 15.0 ... 15.0 15.0 15.0 15.0
    TEMP     (nlat, nlon) float64 15.0 15.0 15.0 15.0 ... 15.0 15.0 15.0 15.0

This synthetic dataset has multiple `X` and `Y` coords. An example would be
model output on a staggered grid.


In [4]:
from cf_xarray.datasets import multiple

multiple

<xarray.Dataset>
Dimensions:  (x1: 30, x2: 10, y1: 20, y2: 5)
Coordinates:
  * x1       (x1) int64 0 1 2 3 4 5 6 7 8 9 10 ... 20 21 22 23 24 25 26 27 28 29
  * y1       (y1) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
  * x2       (x2) int64 0 1 2 3 4 5 6 7 8 9
  * y2       (y2) int64 0 1 2 3 4
Data variables:
    v1       (x1, y1) float64 15.0 15.0 15.0 15.0 15.0 ... 15.0 15.0 15.0 15.0
    v2       (x2, y2) float64 15.0 15.0 15.0 15.0 15.0 ... 15.0 15.0 15.0 15.0

This dataset has ancillary variables


In [5]:
from cf_xarray.datasets import anc

anc

<xarray.Dataset>
Dimensions:            (x: 10, y: 20)
Dimensions without coordinates: x, y
Data variables:
    q                  (x, y) float64 0.9244 0.04556 0.9236 ... 1.077 -1.233
    q_error_limit      (x, y) float64 1.058 -1.969 0.3829 ... -1.01 -1.787
    q_detection_limit  float64 0.001

## What attributes have been discovered?

The criteria for identifying variables using CF attributes are listed
[here](../criteria.rst).


In [6]:
ds.lon

<xarray.DataArray 'lon' (lon: 53)>
array([200. , 202.5, 205. , 207.5, 210. , 212.5, 215. , 217.5, 220. , 222.5,
       225. , 227.5, 230. , 232.5, 235. , 237.5, 240. , 242.5, 245. , 247.5,
       250. , 252.5, 255. , 257.5, 260. , 262.5, 265. , 267.5, 270. , 272.5,
       275. , 277.5, 280. , 282.5, 285. , 287.5, 290. , 292.5, 295. , 297.5,
       300. , 302.5, 305. , 307.5, 310. , 312.5, 315. , 317.5, 320. , 322.5,
       325. , 327.5, 330. ], dtype=float32)
Coordinates:
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
Attributes:
    standard_name:  longitude
    long_name:      Longitude
    units:          degrees_east
    axis:           X

`ds.lon` has attributes `axis: X`. This means that `cf_xarray` can identify the
`'X'` axis as being represented by the `lon` variable.

It can also use the `standard_name` and `units` attributes to infer that `lon`
is "Longitude". To see variable names that `cf_xarray` can infer, use `ds.cf`


In [7]:
ds.cf

Coordinates:
- CF Axes: * X: ['lon']
           * Y: ['lat']
           * T: ['time']
             Z: n/a

- CF Coordinates: * longitude: ['lon']
                  * latitude: ['lat']
                  * time: ['time']
                    vertical: n/a

- Cell Measures:   area, volume: n/a

- Standard Names: * latitude: ['lat']
                  * longitude: ['lon']
                  * time: ['time']

Data Variables:
- Cell Measures:   area, volume: n/a

- Standard Names:   air_temperature: ['air']

For `pop`, only `latitude` and `longitude` are detected, not `X` or `Y`. Please
comment here: https://github.com/xarray-contrib/cf-xarray/issues/23 if you have
opinions about this behaviour.


In [8]:
pop.cf

Coordinates:
- CF Axes: * X: ['nlon']
           * Y: ['nlat']
             Z, T: n/a

- CF Coordinates:   longitude: ['TLONG', 'ULONG']
                    latitude: ['TLAT', 'ULAT']
                    vertical, time: n/a

- Cell Measures:   area, volume: n/a

- Standard Names:   n/a

Data Variables:
- Cell Measures:   area, volume: n/a

- Standard Names:   sea_water_potential_temperature: ['TEMP']
                    sea_water_x_velocity: ['UVEL']

For `multiple`, multiple `X` and `Y` coordinates are detected


In [9]:
multiple.cf

Coordinates:
- CF Axes: * X: ['x1', 'x2']
           * Y: ['y1', 'y2']
             Z, T: n/a

- CF Coordinates:   longitude, latitude, vertical, time: n/a

- Cell Measures:   area, volume: n/a

- Standard Names:   n/a

Data Variables:
- Cell Measures:   area, volume: n/a

- Standard Names:   n/a

## Feature: Accessing coordinate variables

`.cf` implements `__getitem__` to allow easy access to coordinate and axis
variables.


In [10]:
ds.cf["X"]

<xarray.DataArray 'lon' (lon: 53)>
array([200. , 202.5, 205. , 207.5, 210. , 212.5, 215. , 217.5, 220. , 222.5,
       225. , 227.5, 230. , 232.5, 235. , 237.5, 240. , 242.5, 245. , 247.5,
       250. , 252.5, 255. , 257.5, 260. , 262.5, 265. , 267.5, 270. , 272.5,
       275. , 277.5, 280. , 282.5, 285. , 287.5, 290. , 292.5, 295. , 297.5,
       300. , 302.5, 305. , 307.5, 310. , 312.5, 315. , 317.5, 320. , 322.5,
       325. , 327.5, 330. ], dtype=float32)
Coordinates:
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
Attributes:
    standard_name:  longitude
    long_name:      Longitude
    units:          degrees_east
    axis:           X

Indexing with a scalar key raises an error if the key maps to multiple variables
names


In [11]:
multiple.cf["X"]

KeyError: "Receive multiple variables for key 'X': ['x1', 'x2']. Expected only one. Please pass a list ['X'] instead to get all variables matching 'X'."

In [ ]:
pop.cf["longitude"]

To get back all variables associated with that key, pass a single element list
instead.


In [ ]:
multiple.cf[["X"]]

In [ ]:
pop.cf[["longitude"]]

DataArrays return DataArrays


In [ ]:
pop.UVEL.cf["longitude"]

`Dataset.cf[...]` returns a single `DataArray`, parsing the `coordinates`
attribute if present, so we correctly get the `TLONG` variable and not the
`ULONG` variable


In [ ]:
pop.cf["TEMP"]

`Dataset.cf[...]` also interprets the `ancillary_variables` attribute. The
ancillary variables are returned as coordinates of a DataArray


In [ ]:
anc.cf["q"]

## Feature: Accessing variables by standard names


In [ ]:
pop.cf[["sea_water_potential_temperature", "UVEL"]]

Note that ancillary variables are included as coordinate variables


In [ ]:
anc.cf["specific_humidity"]

## Feature: Utility functions

There are some utility functions to allow use by downstream libraries


In [ ]:
pop.cf.keys()

You can test for presence of these keys


In [ ]:
"sea_water_x_velocity" in pop.cf

You can also get out the available Axis names


In [ ]:
pop.cf.axes

or available Coordinate names. Same for cell measures (`.cf.cell_measures`) and
standard names (`.cf.standard_names`).


In [ ]:
pop.cf.coordinates

**Note:** Although it is possible to assign additional coordinates,
`.cf.coordinates` only returns a subset of
`("longitude", "latitude", "vertical", "time")`.


## Feature: Rewriting property dictionaries

`cf_xarray` will rewrite the `.sizes` and `.chunks` dictionaries so that one can
index by a special CF axis or coordinate name


In [ ]:
ds.cf.sizes

Note the duplicate entries above:

1. One for `X`, `Y`, `T`
2. and one for `longitude`, `latitude` and `time`.

An error is raised if there are multiple `'X'` variables (for example)


In [ ]:
multiple.cf.sizes

In [ ]:
multiple.v1.cf.sizes

## Feature: Renaming variables

`cf_xarray` lets you rewrite variables in one dataset to like variables in
another dataset.

In this example, a one-to-one mapping is not possible and the coordinate
variables are not renamed.


In [ ]:
da = pop.cf["TEMP"]
da.cf.rename_like(ds)

If we drop the `X` and `Y` axes, a one-to-one mapping is possible. In this
example, `TLONG` and `TLAT` are renamed to `lon` and `lat` i.e. their
counterparts in `ds`. Note the the `coordinates` attribute is appropriately
changed.


In [ ]:
da = da.cf.drop_vars(["X", "Y"])
da.cf.rename_like(ds)

## Feature: Rewriting arguments

`cf_xarray` can rewrite arguments for a large number of xarray functions. By
this I mean that instead of specifing say `dim="lon"`, you can pass `dim="X"` or
`dim="longitude"` and `cf_xarray` will rewrite that to `dim="lon"` based on the
attributes present in the dataset.

Here are a few examples


### Slicing


In [ ]:
ds.air.cf.isel(T=1)

Slicing works will expand a single key like `X` to multiple dimensions if those
dimensions are tagged with `axis: X`


In [ ]:
multiple.cf.isel(X=1, Y=1)

### Reductions


In [ ]:
ds.air.cf.mean("X")

Expanding to multiple dimensions is also supported


In [ ]:
# takes the mean along ["x1", "x2"]
multiple.cf.mean("X")

### Plotting


In [ ]:
ds.air.cf.isel(time=1).cf.plot(x="X", y="Y")

In [ ]:
ds.air.cf.isel(T=1, Y=[0, 1, 2]).cf.plot(x="longitude", hue="latitude")

`cf_xarray` can facet


In [ ]:
seasonal = (
    ds.air.groupby("time.season")
    .mean()
    .reindex(season=["DJF", "MAM", "JJA", "SON"])
)
seasonal.cf.plot(x="longitude", y="latitude", col="season")

### Resample & groupby


In [ ]:
ds.cf.resample(T="D").mean()

`cf_xarray` also understands the "datetime accessor" syntax for groupby


In [ ]:
ds.cf.groupby("T.month").mean("longitude")

### Rolling & coarsen


In [ ]:
ds.cf.rolling(X=5).mean()

`coarsen` works but everything later will break because of xarray bug
https://github.com/pydata/xarray/issues/4120

`ds.isel(lon=slice(50)).cf.coarsen(Y=5, X=10).mean()`


## Feature: mix "special names" and variable names


In [ ]:
ds.cf.groupby("T.month").mean(["lat", "X"])

## Feature: Weight by Cell Measures

`cf_xarray` can weight by cell measure variables if the appropriate attribute is
set


In [ ]:
# Lets make some weights (not sure if this is right)
ds.coords["cell_area"] = (
    np.cos(ds.air.cf["latitude"] * np.pi / 180)
    * xr.ones_like(ds.air.cf["longitude"])
    * 105e3
    * 110e3
)
# and set proper attributes
ds.air.attrs["cell_measures"] = "area: cell_area"

In [ ]:
ds.air.cf.weighted("area").mean(["latitude", "time"]).cf.plot(x="longitude")
ds.air.mean(["lat", "time"]).cf.plot(x="longitude")

## Feature: Cell boundaries and vertices

`cf_xarray` can infer cell boundaries (for rectilinear grids) and convert
CF-standard bounds variables to vertices.


In [ ]:
ds_bnds = ds.cf.add_bounds(["lat", "lon"])
ds_bnds

We can also convert each bounds variable independently with the top-level
functions


In [ ]:
lat_bounds = ds_bnds.cf.get_bounds("latitude")

lat_vertices = cfxr.bounds_to_vertices(lat_bounds, bounds_dim="bounds")
lat_vertices

In [ ]:
# Or we can convert _all_ bounds variables on a dataset
ds_crns = ds_bnds.cf.bounds_to_vertices()
ds_crns